In [ ]:
# Import Libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains

#Enter Twitter

url ="https://www.twitter.com"

driver = webdriver.Chrome()

driver.get(url)

time.sleep(3)

#Sign in

sign_in = driver.find_element(By.CSS_SELECTOR, "[data-testid='loginButton']")

sign_in.click()

time.sleep(3)

#Input Email
# Localiza el campo de texto por su atributo autocomplete

user_field = driver.find_element(By.CSS_SELECTOR, "input[autocomplete='username']")

# Introduce el nombre de usuario
username = "enter your email here"
user_field.send_keys(username)

time.sleep(3)

element = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]")

time.sleep(5)

element.click()

#Input Username

time.sleep(3)
# Localiza el elemento por su selector CSS
user_bar = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div[2]/label/div/div[2]/div/input")

username = "enter your username here"
# Introduce el texto en el elemento
user_bar.send_keys(username)
next_bt = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div/div/div/div/span/span")
next_bt.click()

#Input Password

time.sleep(3)

# Localiza el elemento por su selector CSS
campo_contraseña = driver.find_element(By.CSS_SELECTOR, "input[name='password']")

password = "enter your password here"
# Introduce el texto en el campo de contraseña
campo_contraseña.send_keys(password)

next_bt = driver.find_element(By.XPATH, "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div/span/span")

next_bt.click()
time.sleep(3)

search_filed = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input")

search_user = "enter the target user here" #it has to be with the @
search_filed.send_keys(search_user)

time.sleep(2)

search_bt = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[2]/div/div[5]/div/div")

search_bt.click()
time.sleep(5)

time_elements = driver.find_elements(By.XPATH, '//time[contains(@datetime, "T") and contains(text(), " ") and not(ancestor::*[@class="css-175oi2r r-eqz5dr r-1fz3rvf r-1s2bzr4"])]')

form_elemets = []
years = []
months = []
days = []
hours_raw = []


for elemento in time_elements:


    fecha_y_hora = elemento.get_attribute("datetime").split("T")
    fecha = fecha_y_hora[0]
    dia = fecha.split("-")[2]
    mes = fecha.split("-")[1]
    anno = fecha.split("-")[0]
    texto_formateado = f"{dia} {mes}. {anno}"
    elemento_formateado = f'<time datetime="{fecha_y_hora[0]}T{fecha_y_hora[1]}">{texto_formateado}</time>'
    form_elemets.append(elemento_formateado)
    years.append(anno)
    months.append(mes)
    days.append(dia)
    hours_raw.append(fecha_y_hora[1])


hours = [hour.split('.')[0] for hour in hours_raw]
for elemento_formateado in form_elemets:
    print(elemento_formateado)
    
years
months
days
hours_raw
hours
